# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
2. Công cụ sử dụng
3. Các bước thu thập dữ liệu

In [44]:
# import những  thư viện cần thiết
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [45]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [46]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [47]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()

for i in range(1,50):
   # Khởi tạo url vầ yêu cầu lấy page về 
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}'.format(i)
   
   print(url)
   
   page_content = get_content(url, headers)
   
   soup = BeautifulSoup(page_content.text, 'html.parser')
   
   containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
   print('Đã tìm thấy {} công việc'.format(len(containers)))
   for container in containers:
      dict_job = {}
      
      buffer = BASE_URL + container.find('a', href=True)['href']
      dict_job['Liên kết'] = buffer if buffer else None
      
      buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
      dict_job['Tên công việc'] = buffer.text if buffer else None
      
      buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
      dict_job['Tên công ty'] = buffer.text if buffer else None
      
      buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
      dict_job['Mức lương'] = buffer.text if buffer else None
      
      buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
      dict_job['Khu vực tuyển'] = buffer.text if buffer else None
      
      job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)
         


https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=8
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=9
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=10
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=11
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
183,https://vieclam24h.vn/ke-toan/ke-toan-noi-bo-h...,Kế Toán Nội Bộ (Hoàn Kiếm - Hà Nội),Chi Nhánh - Công Ty TNHH Cà Phê Là Việt,8 - 12 triệu,Hà Nội
519,https://vieclam24h.vn/nhan-su/nhan-vien-hanh-c...,Nhân Viên Hành Chính Nhân Sự,Công Ty TNHH Quốc Tế Milwee Việt Nam,8 - 10 triệu,Hà Nội
77,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Kinh Doanh / Telesale / Bán Hàng Qua...,CÔNG TY TNHH MTV HEALTHPOST,15 - 25 triệu,TP.HCM
1161,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,"Nhân Viên Chăm Sóc Khách Hàng (Quận Tân Bình, ...",Công Ty TNHH Dịch Thuật Và Phiên Dịch Chuyên N...,7 - 15 triệu,TP.HCM
500,https://vieclam24h.vn/ke-toan/truong-phong-ke-...,Trưởng Phòng Kế Toán,Công Ty TNHH Tư Vấn Và Xây Dựng Xml,25 - 60 triệu,TP.HCM


In [58]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [49]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
26,https://vieclam24h.vn/van-hanh-bao-tri-bao-duo...,Quản Lý Về Kỹ Thuật Bảo Hành Và Sửa Chữa Kho V...,Công Ty CP Phân Phối Quốc Tế Genex,20 - 35 triệu,Hà Nội
1125,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Hỗ Trợ Thanh Toán Địa Bàn Khu Vực Lo...,Công ty Tài Chính TNHH HD SAISON,15 - 20 triệu,Long An
993,https://vieclam24h.vn/ban-hang-kinh-doanh/nu-n...,Nữ Nhân Viên Kinh Doanh Thực Phẩm - Thu Nhập T...,24h Seafood - Công Ty TNHH 24h Seafood,10 - 15 triệu,Hà Nội
357,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên Pha Chế - Phan Thiết,Công Ty Cổ Phần Tập Đoàn Đầu Tư Địa Ốc No Va,6 - 6.5 triệu,Bình Thuận
635,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Tư Vấn Tín Dụng - Kênh Trả Góp - Thu...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Bình Dương


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       1470 non-null   object
 1   Tên công việc  1470 non-null   object
 2   Tên công ty    1470 non-null   object
 3   Mức lương      1470 non-null   object
 4   Khu vực tuyển  1470 non-null   object
dtypes: object(5)
memory usage: 57.6+ KB


In [52]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])
for url in df['Liên kết']:
    job_content = get_content(url, headers=headers)
    if hasattr(job_content, 'text'):
        job_content = job_content.text
    else:
        continue
    
    soup = BeautifulSoup(job_content, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = soup.find('a', class_='hover:text-se-accent')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None


    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

    raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!
Success!!!

In [53]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
375,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Công Ty Cổ Phần Công Nghệ Giáo Dục Edtexco,Nhân Viên Kinh Doanh - Chuyên Viên Tư Vấn,10 - 20 triệu,Hà Nội,Chuyên viên- nhân viên,5,Toàn thời gian cố định,Cao đẳng,1 năm,Chăm sóc khách hàng/Bán hàng - Kinh doanh/Giáo...,"Hà Nội, Tầng 9, Tòa nhà Zodiac, Số 1, Ngõ 19, ...","- Giới thiệu, tư vấn giải pháp cho khách hàng ...",- Tốt nghiệp từ cao đẳng trở lên các chuyên ng...,- Lương cứng thưởng theo hiệu quả kinh doanh u...,Việc làm Hà NộiChăm sóc khách hàngCông Ty Cổ P...,"Tầng 9, Tòa nhà Zodiac, Số 1, Ngõ 19, Phố Duy ...",10 - 150 người
1087,https://vieclam24h.vn/ban-hang-kinh-doanh/nhu-...,Công ty Tài Chính TNHH HD SAISON,"Như Thanh, Như Xuân - Thanh Hóa: Nhân Viên Hỗ ...",15 - 30 triệu,Thanh Hóa,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Trung học,Dưới 1 năm,Bán hàng - Kinh doanh/Tài chính - Đầu tư - Chứ...,"Thanh Hóa, Thanh Hóa, Quan HóaThanh Hóa, Thanh...",1. Liên hệ trực tiếp với khách hàng trong...,"- Tốt nghiệp THPT- Sẵn sàng di chuyển, am ...","Lương tháng 13, thưởng theo thâm niên.Ký Hợp đ...",toánhóabànViệc làm Thanh HóaBán hàng - Kinh do...,"Tầng 3, tòa nhà VIT, 519 Kim Mã, Ba Đình, Hà Nội",Trên 300 người
99,https://vieclam24h.vn/ke-toan/nhan-vien-thu-ng...,Công Ty Cổ Phần Xuất Nhập Khẩu Hạt Ngọc Việt,Nhân Viên Thu Ngân,8 - 10 triệu,TP.HCM,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Trung cấp,1 năm,Kế toán/Thu mua - Kho Vận - Chuỗi cung ứng,"TP.HCM, Tầng 2, Rosana Tower, Số 60 Nguyễn Đìn...","• Thực hiện quy trình thu ngân (Nhập order, th...","• Có kinh nghiệm ít nhất 1 năm• Trung thực, nh...",• Mức lương: 8 - 10 triệu/ tháng tùy kinh nghi...,Việc làm TP.HCMKế toánCông Ty Cổ Phần Xuất ...,"Tầng 2, Rosana Tower, Số 60 Nguyễn Đình Chiểu,...",10 - 150 người
1033,https://vieclam24h.vn/ban-hang-kinh-doanh/thua...,Công ty Tài Chính TNHH HD SAISON,Thừa Thiên Huế - Nhân Viên Hỗ Trợ Thanh Toán T...,15 - 30 triệu,Thừa Thiên Huế,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Trung học,Chưa có kinh nghiệm,Bán hàng - Kinh doanh/Tài chính - Đầu tư - Chứ...,"Thừa Thiên Huế, Hương Trà",- Liên hệ trực tiếp với khách hàng tron...,"- Tốt nghiệp THPT- Sẵn sàng di chuyển, am ...",- Thử việc 100% lương;- Ký hợp đ...,bànthanh toánquận bình thạnh thành phố hồ chí ...,"Tầng 8, 9, 10, Tòa Nhà Gilimex, 24C Phan Đăng ...",Trên 300 người
1106,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty Cổ Phần Ô Tô Vũ Linh,Nhân Viên Kinh Doanh Ô Tô Thu Nhập Upto 50 Triệu,20 - 50 triệu,"Bắc Ninh,",Chuyên viên- nhân viên,5,Toàn thời gian cố định,None,1 năm,Bán hàng - Kinh doanh/Chăm sóc khách hàng/Vận ...,"Bắc Ninh, Từ SơnHà Nội, 40 Ngô Gia Tự, Long Biên","- Tư vấn, giới thiệu và bán các loại Ô Tô, xe ...","- Nam/Nữ, Ngoại hình ưa nhìn.- Không yêu cầu b...",- Thu nhập: Lương cứng lên tới 8tr + hoa hồng ...,nhậpquận long biênvisual merchandisingViệc làm...,"Số 40 đường Ngô Gia Tự, Phường Đức Giang, Quận...",10 - 150 người


In [57]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             1 non-null      object
 1   Tên công ty          1 non-null      object
 2   Tên công việc        1 non-null      object
 3   Mức lương            1 non-null      object
 4   Khu vực tuyển        1 non-null      object
 5   Cấp bậc              1 non-null      object
 6   Số lượng tuyển       1 non-null      object
 7   Hình thức làm việc   1 non-null      object
 8   Yêu cầu bằng cấp     1 non-null      object
 9   Yêu cầu kinh nghiệm  1 non-null      object
 10  Ngành nghề           1 non-null      object
 11  Địa điểm làm việc    1 non-null      object
 12  Mô tả công việc      1 non-null      object
 13  Yêu cầu công việc    1 non-null      object
 14  Quyền lợi            1 non-null      object
 15  Từ khóa              1 non-null      object
 16  Địa chỉ công

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [55]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}
    
    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None
    
    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None
    
    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None
    
    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None
    
    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên CSKH - Ứng Dụng Du Lịch Tiếng Anh (Q...,Công Ty TNHH Vietnam Concentrix Services,10 - 15 triệu,TP.HCM
1,https://vieclam24h.vn/hanh-chinh-thu-ky/bien-p...,Biên Phiên Dịch Tiếng Trung Tại Long An,Công Ty Cổ Phần Gia Phú,10 - 20 triệu,Long An
2,https://vieclam24h.vn/ban-hang-kinh-doanh/tro-...,Trợ Lý Phát Triển Kinh Doanh,Công Ty Cổ Phần Giấy CP,12 - 14 triệu,TP.HCM
3,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Telesales (Chợ Tốt Xe),Công Ty TNHH Chợ Tốt Trading,6 - 10 triệu,Hà Nội
4,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Nhân Viên Dịch Vụ Khách Hàng,Mcredit - Công Ty Tài Chính TNHH Mb Shinsei,10 - 20 triệu,Hà Nội
5,https://vieclam24h.vn/ban-hang-kinh-doanh/chuy...,Chuyên Viên Tư Vấn Tài Chính Hồ Chí Minh (Lươn...,Công Ty TNHH Bảo Hiểm Nhân Thọ AIA Việt Nam,16 - 30 triệu,TP.HCM
6,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Kinh Doanh Nội Thất - Thu Nhập 20-30...,Công Ty TNHH Kiến Trúc Milan,20 - 30 triệu,Hà Nội
7,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Trợ Lý (Biết Tiếng Trung),Công Ty TNHH Việt Nam Paiho,7 - 15 triệu,TP.HCM
8,https://vieclam24h.vn/ke-toan/ke-toan-truong-c...,Kế Toán Trưởng,Công Ty TNHH Thương Mại Dịch Vụ Phát Triển Sản...,18 - 30 triệu,TP.HCM
9,https://vieclam24h.vn/ngan-hang/hcm-giam-doc-q...,HCM - Giám Đốc Quan Hệ Khách Hàng Ưu Tiên,Acb - Ngân Hàng Thương Mại Cổ Phần Á Châu,16 - 50 triệu,TP.HCM


In [ ]:

# job = get_content('https://vieclam24h.vn/hanh-chinh-thu-ky/bien-phien-dich-tieng-trung-tai-long-an-c1p123id200290693.html',headers= headers)

# with open('../data/job_content.txt', 'w', encoding='utf-8') as f:
#     f.write(str(job.text))

In [56]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển', 
# Thông tin chung    
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

# with open('../data/job_content.txt', 'r', encoding='utf-8') as f:
#     job_content = f.read()
url = 'https://vieclam24h.vn/xay-dung/chi-huy-truong-cong-trinh-c31p73id200288561.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

dict_job = {}

dict_job['Liên kết'] = 'https://vieclam24h.vn/ban-hang-kinh-doanh/'

buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
dict_job['Tên công ty'] = buffer.text if buffer else None

buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
dict_job['Tên công việc'] = buffer.text if buffer else None

buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
dict_job['Mức lương'] = buffer.text if buffer else None

buffer = soup.find('a', class_='hover:text-se-accent')
dict_job['Khu vực tuyển'] = buffer.text if buffer else None


buffer = soup.find('p', text="Cấp bậc")
dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Số lượng tuyển")
dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Hình thức làm việc")
dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu bằng cấp")
dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu kinh nghiệm")
dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Ngành nghề")
dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


buffer1 = soup.find('h4', text="Mô tả công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Yêu cầu công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Quyền lợi")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Địa điểm làm việc")
buffer2 = buffer1.find_next_sibling('ul')
dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None 


buffer1 = soup.find('h3', text="Từ khoá")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Từ khóa'] = buffer2.text if buffer2 else None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
for buffer in buffers:
    if buffer.get_text() == 'Địa chỉ:':
        dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
    if buffer.get_text() == 'Quy mô:':
        dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


Success!!!


C:\Users\Admin\AppData\Local\Temp\ipykernel_5464\539015741.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Cấp bậc")
C:\Users\Admin\AppData\Local\Temp\ipykernel_5464\539015741.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Số lượng tuyển")
C:\Users\Admin\AppData\Local\Temp\ipykernel_5464\539015741.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Hình thức làm việc")
C:\Users\Admin\AppData\Local\Temp\ipykernel_5464\539015741.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Yêu cầu bằng cấp")
C:\Users\Admin\AppData\Local\Temp\ipykernel_5464\539015741.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecate

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
0,https://vieclam24h.vn/ban-hang-kinh-doanh/,Công Ty CP Hóa Chất Xây Dựng Bách Khoa,Chỉ Huy Trưởng Công Trình,13 - 17 triệu,Hà Nội,Quản lý cấp trung,2,Toàn thời gian cố định,Đại học,2 năm,Xây dựng/Quản lý dự án,"Hà Nội, Phòng 2904, tháp A, chung cư thăng lon...",- Điều phối chung các công việc tại dự án.- Là...,- Tốt nghiệp đại học- Có chứng chỉ giám sát hi...,- Lương: Cạnh tranh theo năng lực (Thu nhập : ...,Việc làm Hà NộiXây dựngCông Ty CP Hóa Chất ...,"Phòng 2904, tháp A, chung cư thăng long Number...",10 - 150 người
